In [1]:
import sys
sys.path.append("../src")

import pandas as pd

import plotly.graph_objs as go
import plotly.offline as py

import utilities as u
import constants as c

In [2]:
df_l = pd.read_excel("../data/data.xlsx", "liquid_m", index_col=0)
# Add total column
if c.names.TOTAL in df_l.columns:
    del df_l[c.names.TOTAL]

df_l[c.names.TOTAL] = df_l.sum(axis=1)

df_t = pd.read_excel("../data/data.xlsx", "trans_m", index_col=0)
df_t = u.dfs.fix_df_trans(df_t)

In [12]:
def plot_expenses_vs_liquid(df_trans_in, df_liquid_in, avg_month=12):
    
    df_l = df_liquid_in.rolling(avg_month, min_periods=1).mean()
    df_t = u.dfs.group_df_by(df_trans_in[df_trans_in[c.cols.TYPE] == c.names.EXPENSES], "M")
    df_t = df_t.rolling(avg_month, min_periods=1).mean()

    iter_data = [
        (df_t, df_t[c.cols.AMOUNT], c.names.EXPENSES, c.colors.EXPENSES),
        (df_t, 3*df_t[c.cols.AMOUNT], c.names.LIQUID_MIN_REC, c.colors.LIQUID_MIN_REC),
        (df_t, 6*df_t[c.cols.AMOUNT], c.names.LIQUID_REC, c.colors.LIQUID_REC),
        (df_l, df_l[c.names.TOTAL], c.names.LIQUID, c.colors.LIQUID),
    ]
    
    data = [go.Scatter(x=df.index, y=y, name=name, marker={"color": color}) for df, y, name, color in iter_data]
    
    df_aux = df_l[c.names.TOTAL]/df_t[c.cols.AMOUNT]
    data.append(go.Scatter(x=df_aux.index, y=df_aux, name="Months of expenses", marker={"color": "yellow"}, yaxis='y2'))

    layout = go.Layout(title="Liquid vs Expenses", yaxis={"title": 'yaxis title'},
        yaxis2={
            "title": 'yaxis2 title',
            "overlaying": 'y',
            "side": 'right'
        }
        )
    
    py.plot(go.Figure(data=data, layout=layout))
    
plot_expenses_vs_liquid(df_t, df_l)

In [ ]:
c.colors.L